In [ ]:
import numpy as np
import pickle
import pandas as pd
import datetime
import re
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,normalize
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [ ]:
%pwd

'/content/drive/MyDrive/Kaggle'

In [ ]:
def final(page,date):
  date1=datetime.date(2017,7,9)
  k=date.split("-")
  date2=datetime.date(int(k[0]),int(k[1]),int(k[2]))
  date_diff=(date2-date1).days
  data=pd.read_csv("final.csv")
  index=data[data["Page"]==page].index.values
  k =np.array(data.iloc[index[0]].values[-264:], dtype=int) 
  x_pred=k[:200] 
  y_true=k[200:264]
  x_pred=np.log1p(x_pred)
  access=[]
  agent=[]
  language=[]
  temp=page.split(".")
  k=temp[-3].split("_")
  if k[-1]=="commons" or k[-1]=="www":
    language.append("media")
  else:
    language.append(k[-1])
  t=temp[-1].split("_")
  access.append(t[1])
  agent.append(t[2])
  with open('enc_access','rb') as file:
    access_enc=pickle.load(file) 

  with open('enc_language','rb') as file:
    lang_enc=pickle.load(file)

  with open('enc_agent','rb') as file:
    agent_enc=pickle.load(file)
  access_ohe=access_enc.transform([access]).reshape(1,1)
  agent_ohe=agent_enc.transform([agent]).reshape(1,1)
  language_ohe=lang_enc.transform([language]).reshape(1,1)
  model=load_model("lstm.h5")
  x_pred=np.array(x_pred).reshape(1,200,1)
  pred=model.predict([x_pred,access_ohe,language_ohe,agent_ohe])
  pred=np.expm1(pred)
  return agent[0],access[0],language[0],int(pred[0][date_diff]),y_true[date_diff]

In [ ]:
x = PrettyTable()
start=datetime.datetime.now()
agent,access,language,predicted,y_true=final("2PM_zh.wikipedia.org_all-access_spider","2017-7-20")
x = PrettyTable(["agent","Access", "Language",'Predicted','Time Taken'])
x.add_row([agent,access,language,predicted,datetime.datetime.now()-start])
print(x)

+--------+------------+----------+-----------+----------------+
| agent  |   Access   | Language | Predicted |   Time Taken   |
+--------+------------+----------+-----------+----------------+
| spider | all-access |    zh    |     16    | 0:00:06.490480 |
+--------+------------+----------+-----------+----------------+


In [ ]:
def final_2(set):
  x = PrettyTable()
  x = PrettyTable(["agent","Access", "Language",'Predicted','Actual','SMAPE','Time Taken'])
  for i in set:
    start=datetime.datetime.now()
    agent,access,language,predicted,actual=final(i[0],i[1])
    smape=np.abs(actual-predicted)/((actual+predicted)/2)
    x.add_row([agent,access,language,predicted,actual,np.round(smape,3),datetime.datetime.now()-start])
  print(x)

In [ ]:
final_2([["5566_zh.wikipedia.org_all-access_spider","2017-8-20"],
         ["Intel_80386_zh.wikipedia.org_all-access_spider","2017-8-10"],
         ["Netflix_zh.wikipedia.org_all-access_spider","2017-7-29"],
         ["2PM_zh.wikipedia.org_all-access_spider","2017-7-20"]])

+--------+------------+----------+-----------+--------+-------+----------------+
| agent  |   Access   | Language | Predicted | Actual | SMAPE |   Time Taken   |
+--------+------------+----------+-----------+--------+-------+----------------+
| spider | all-access |    zh    |     19    |   14   | 0.303 | 0:00:05.913324 |
| spider | all-access |    zh    |     8     |   7    | 0.133 | 0:00:05.820181 |
| spider | all-access |    zh    |     29    |   25   | 0.148 | 0:00:05.762735 |
| spider | all-access |    zh    |     16    |   15   | 0.065 | 0:00:06.031186 |
+--------+------------+----------+-----------+--------+-------+----------------+
